In [1]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using DataFrames
using CSV
using JLD
using MySQL
using Distributions
using ThreadSafeDicts
using Printf
using MLJ
using MLJBase
using MLDataPattern
using ProgressMeter
using StatsBase

In [2]:
netsurf = CSV.read("../data/netsurf.csv", DataFrame) # read dataset
select!(netsurf, Not(" cb513_mask")) # remove column
for i in ["input", " dssp3", " dssp8"] # remove white spaces in strings
    netsurf[!, i] = [join(map(x -> isspace(a[x]) ? "" : a[x], 1:length(a))) for a in netsurf[!, i]]
end

In [18]:
k25 = vcat(
    JLD.load("../data/data_netsurf/k250.jld")["k25[ranges[i]:ranges[i + 1]]"], 
    [JLD.load(@sprintf("../data/data_netsurf/k25%s.jld", i))["k25[ranges[i] + 1:ranges[i + 1]]"] for i in 1:5]...)

1201-element Vector{Any}:
 Any[Bool[1, 1, 0, 0, 0, 0, 0, 0, 1, 0  …  1, 1, 0, 1, 1, 0, 0, 0, 0, 0], Bool[0, 0, 1, 0, 0, 0, 0, 1, 0, 1  …  1, 0, 0, 0, 0, 0, 1, 1, 0, 0], Bool[0, 1, 0, 1, 1, 1, 0, 0, 0, 0  …  1, 0, 1, 0, 0, 0, 0, 1, 0, 1], Bool[1, 1, 0, 0, 1, 0, 0, 0, 0, 0  …  0, 0, 0, 1, 0, 0, 0, 0, 0, 1], Bool[1, 0, 0, 1, 1, 0, 1, 0, 1, 0  …  0, 0, 1, 0, 1, 0, 0, 0, 1, 1], Bool[0, 0, 1, 0, 1, 0, 1, 1, 1, 0  …  1, 1, 1, 1, 0, 0, 1, 0, 1, 0], Bool[1, 0, 1, 0, 1, 0, 0, 0, 1, 0  …  1, 0, 0, 1, 1, 1, 1, 0, 0, 1], Bool[0, 0, 0, 1, 1, 1, 0, 1, 0, 1  …  1, 0, 1, 1, 1, 0, 1, 0, 0, 0], Bool[0, 0, 1, 1, 0, 0, 0, 1, 1, 0  …  0, 0, 1, 1, 0, 0, 1, 1, 1, 0], Bool[0, 1, 0, 0, 1, 1, 0, 1, 0, 1  …  1, 0, 0, 1, 0, 0, 1, 0, 0, 1]  …  Bool[0, 1, 1, 1, 1, 0, 1, 0, 1, 1  …  0, 1, 0, 1, 0, 0, 1, 1, 1, 1], Bool[1, 0, 0, 0, 1, 0, 0, 0, 0, 0  …  0, 1, 0, 1, 0, 0, 1, 0, 0, 1], Bool[0, 1, 0, 0, 1, 1, 1, 0, 0, 0  …  0, 0, 1, 0, 0, 0, 0, 1, 0, 0], Bool[1, 0, 1, 0, 0, 0, 1, 1, 0, 0  …  0, 0, 0, 1, 0, 1, 0, 1, 1, 0], 

In [3]:
k25 = JLD.load("../data/data_netsurf/k250.jld")["k25[ranges[i]:ranges[i + 1]]"]

201-element Vector{Any}:
 Any[Bool[1, 1, 0, 0, 0, 0, 0, 0, 1, 0  …  1, 1, 0, 1, 1, 0, 0, 0, 0, 0], Bool[0, 0, 1, 0, 0, 0, 0, 1, 0, 1  …  1, 0, 0, 0, 0, 0, 1, 1, 0, 0], Bool[0, 1, 0, 1, 1, 1, 0, 0, 0, 0  …  1, 0, 1, 0, 0, 0, 0, 1, 0, 1], Bool[1, 1, 0, 0, 1, 0, 0, 0, 0, 0  …  0, 0, 0, 1, 0, 0, 0, 0, 0, 1], Bool[1, 0, 0, 1, 1, 0, 1, 0, 1, 0  …  0, 0, 1, 0, 1, 0, 0, 0, 1, 1], Bool[0, 0, 1, 0, 1, 0, 1, 1, 1, 0  …  1, 1, 1, 1, 0, 0, 1, 0, 1, 0], Bool[1, 0, 1, 0, 1, 0, 0, 0, 1, 0  …  1, 0, 0, 1, 1, 1, 1, 0, 0, 1], Bool[0, 0, 0, 1, 1, 1, 0, 1, 0, 1  …  1, 0, 1, 1, 1, 0, 1, 0, 0, 0], Bool[0, 0, 1, 1, 0, 0, 0, 1, 1, 0  …  0, 0, 1, 1, 0, 0, 1, 1, 1, 0], Bool[0, 1, 0, 0, 1, 1, 0, 1, 0, 1  …  1, 0, 0, 1, 0, 0, 1, 0, 0, 1]  …  Bool[0, 1, 1, 1, 1, 0, 1, 0, 1, 1  …  0, 1, 0, 1, 0, 0, 1, 1, 1, 1], Bool[1, 0, 0, 0, 1, 0, 0, 0, 0, 0  …  0, 1, 0, 1, 0, 0, 1, 0, 0, 1], Bool[0, 1, 0, 0, 1, 1, 1, 0, 0, 0  …  0, 0, 1, 0, 0, 0, 0, 1, 0, 0], Bool[1, 0, 1, 0, 0, 0, 1, 1, 0, 0  …  0, 0, 0, 1, 0, 1, 0, 1, 1, 0], B

In [7]:
k25 = [convert(Vector{Float32}, x) for i in k25 for x in i]
n_seq = length(k25)
for i in 1:n_seq
    k25[i][k25[i].== 0] .= -1
end
y = first([x for i in netsurf[!, " dssp8"] for x in i], n_seq)
countmap(y)

Dict{Char, Int64} with 8 entries:
  'B' => 455
  'I' => 258
  'H' => 14302
  'E' => 9641
  'G' => 1668
  'S' => 3837
  'T' => 4825
  'C' => 10706

In [8]:
function findmax(d)
    maxkey, maxvalue = 0, -1
    for (key, value) in d
      if value > maxvalue
        maxkey = key
        maxvalue = value
      end
    end
    return maxkey
  end

findmax (generic function with 1 method)

In [24]:
function bruh(x_data1, y_data1, lr = 0.035, k = 10)
    x_data = x_data1
    y_data = y_data1

    stratified_cv = StratifiedCV(;nfolds=k, shuffle = true)
    pairs = MLJBase.train_test_pairs(stratified_cv, 1:length(y_data), y_data)

    classes = collect(Set(y_data))
    n_class = length(classes)

    enu_class = Dict(classes .=> [i for i in 1:n_class])
    println(enu_class)
    rever_enu =  Dict(value => key for (key, value) in enu_class)
    println(rever_enu)
    f1 = Dict(classes .=> zeros(n_class))

    for i in pairs
        train = i[1]
        test = i[2]

        class_hdv = Dict()
                
        classes2 = copy(classes)
        while length(classes2) > 0
            initer = rand(train)
            t = y_data[initer]
            if t ∈ classes2
                class_hdv[t] = x_data[initer]
                filter!(!=(initer), train)
                filter!(!=(t), classes2)
            end
        end
        for j in train #training
            cosines = Dict()
            realcls = y_data[j]

            hdv = x_data[j]
            for c in classes
                cosines[c] = cosine(hdv, class_hdv[c])
            end
            pred_cls = findmax(cosines)
            if realcls != pred_cls
                class_hdv[realcls] = class_hdv[realcls] .+ (lr*(cosines[realcls]) .* hdv)
                class_hdv[pred_cls] = class_hdv[pred_cls] .- (lr*(cosines[pred_cls]) .* hdv)
            end
        end
        #testing
        confusion_matrix = zeros(n_class,n_class)
        for j in 1:length(test) #testing
            cosines = Dict()
            realcls = y_data[j]

            hdv = x_data[j]
            for c in classes
                cosines[c] = cosine(hdv, class_hdv[c])
            end
            pred_cls = findmax(cosines)
            confusion_matrix[enu_class[pred_cls], enu_class[realcls]] += 1
        end

        #evaluating
        for c in 1:n_class
            TP = confusion_matrix[c,c]
            FPp = sum(confusion_matrix[c, :])
            FNp = sum(confusion_matrix[:, c])

            prec = TP/FPp
            recall = TP/FNp

            f1[rever_enu[c]] += 2 * (prec*recall)/(prec+recall)
        end
    end

    for (key,value) in f1
        f1[key] = f1[key] / k
    end
    
    return f1

end

bruh (generic function with 3 methods)

In [27]:
br = bruh(k25, y, 0.00000001)

Dict('B' => 8, 'I' => 5, 'H' => 3, 'E' => 7, 'G' => 4, 'S' => 1, 'T' => 

6, 'C' => 2)
Dict(5 => 'I', 4 => 'G', 6 => 'T', 7 => 'E', 2 => 'C', 8 => 'B', 3 => 'H', 1 => 'S')


Dict{Char, Float64} with 8 entries:
  'B' => 0.0193408
  'I' => NaN
  'H' => 0.20772
  'E' => 0.158228
  'G' => 0.055052
  'S' => 0.100561
  'T' => 0.132702
  'C' => 0.17463